# Multi-Model Integration: Using Gemini, DeepSeek & Grok with OpenAI Agents

#### Models other than OpenAI 
Using OpenAI SDK to drive Gemini and DeepSeek   
#### Structured Outputs 
The way that we can require an agent not to respond jut with text but to populate some kind of an object where we can specify the fields that are going to be populated. 
#### Guardrails 
Which are an approach to making sure that we have some controls over the information that comes in to our agent setup and what comes out

In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)

True

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


Three different sales agents.      
One is professional, one is witty and engaing and one is concise. 

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

We are using OpenAI endpoints.       
We can use compatible OpenAI endpoints to talk to other models like Gemini, DeepSeek and Grok      

In [5]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/opena+i/v1"

We create a new instance of the client by instantiating an async OpenAI object, passing in the base URL and API Key      
      
Finally, we create three model objects       
      
This is little bit of boilerplate, if you are using OpenAI's models then you don't need to do any of this.

In [6]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

When we create our agents, we're going to give them a name and instructions      
If it is OpenAI model. It would be like "model='GPT-4o-mini'" instead of "model=deepseek_model"

In [7]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model=deepseek_model)
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model=gemini_model)
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model=llama3_3_model)

Write a cold sales email and we are now going to repackage each of these three sales agents into three tools

In [8]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

We have a normal function tool.    
We use this decorator to reconstitute wrap this function in the boilerplate JSON that describes the tool

In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("rsolo@ranaa.ca")  # Change to your verified sender
    to_email = To("ranjitsolo@yahoo.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

Subject for a cold sales email.     
Convert a text to HTML email.   
passing in the string GPT-4o-mini      
     
So this is an agent that can write the subject of an email 
An agent that can convert an email to HTML format and 

In [10]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

We collect together those three tools.

In [11]:
email_tools = [subject_tool, html_tool, send_html_email]

email_agent which is able to take some instructions and handoff description

In [12]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

Three sales tools into one group of tools.     
The handoff is now "email_agent"

In [13]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

sales_agent2 - Gemini Sales Agent and sales_agent3 - Llama3.3 Sales Agent didn't work.      
Solution suggested was to update the opanai-agent package to verion 0.0.14        
It fixed the Gemini but not the Llama3.3       
        
uv pip install 'openai-agent==0.0.14'      

In [14]:
# sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
# You never generate sales emails yourself; you always use the tools. \
# You try all 3 sales agent tools at least once before choosing the best one. \
# You can use the tools multiple times if you're not satisfied with the results from the first try. \ # deleted this line
# You select the single best email using your own judgement of which email will be most effective. \
# After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

https://www.udemy.com/course/the-complete-agentic-ai-engineering-course/learn/lecture/49820721#questions/23327033

Hopefully this helps others as well. I fixed the issue by updating openai-agents. Turns out I was using 0.0.8 (that was just 2 weeks ago) and there have been multiple releases already - see https://pypi.org/project/openai-agents/#history

I'm using 0.0.14 now and the problem went away.

pyproject.toml => update openai-agents version to 0.0.14

Submitted PR https://github.com/ed-donner/agents/pull/9 as well, hopefully others won't run into this like me.



Here're some other things I have tried but without luck, just sharing some steps I took. :)

- The JSON 'strict' field was set to true for the sales agents but I think that is irrelevant for external validation (e.g. GEMINI, GROQ) and only applies to openai SDK payload itself.

- Adding instructions in the 'instructionsX' to format/remove additional JSON fields didn't work for me as well. Because the payload is strictly following OpenAI SDK JSON format and schema, it doesn't remove it via the prompt/instructions.

- I reviewed the https://ai.google.dev/gemini-api/docs/openai documents, and can't tell anything, that's when it struck me about the python module "openai-agents", and I took a look at https://github.com/openai/openai-agents-python, and finally noticed the same issue others had encountered https://github.com/openai/openai-agents-python/issues/443



There you go, hopefully the steps here can also guide others to troubleshoot some of the issues you have encountered. Good luck!

Anthropic's claude is not compatible but it can be used via OpenRouter

## Check out the trace:

https://platform.openai.com/traces

Next, Structured Output and Guardrails        
Guardrails are ways that you can put a constraint around your agent platform.      
It's a test to check the results. If correct response was provided.     
Guardrails can themselves be agents, which means tht you use an LLM to be checking that things look good at any point in your flow.      
Actually, guardrails can only be applied either to the input at the very beginning of the first agent or output of the last agent.     
It is designed to protect your model against getting an input, which is inappropriate or not intentded and also producing an output not acceptable.      

We will be implementing the Guardrail for the input.      
     
Structured output is defined in a class `NameCheckOutput`. This is a pydantic object where you have objects classes which are designed to reflect a particular schema of data.      
The class has two fields, 'is_name_in_message' and 'name'        
      
Then Guardrail agent, it has a name, instruction to check if there are any personal name, and structured output      
(we don't want the output to be text but structured output conforms to the schema)        
       
We could use this so that rather than emails being just strings, they could actually have a subject, a recipient, a body     

In [15]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

So, Guardrails is like an asynchronous, it is a coroutine       
you simply decorate it with @input_guardrail      
     
There is one for @output_guardrail        
      
It is going to check if someone's personal name is being included in the message and then return `GuardrailFunctionOutput`       
It has two things, 'output_info' dictionary useful for tracing and 'tripwire_trigger' is a boolean. If there is a problem (True) or not(False)       
     
If there is a name, then it should trigger the tripwire and the guardrail to fail

In [16]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

`careful_sales_manager` is a new agent, everything is same as before, the difference is we are passing in an input guardrail       
     
Note: 'Alice' in the message

In [17]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

## Check out the trace:

https://platform.openai.com/traces

Trace > guardrail_against_name > Name check > POST/v1/responses      
       
Output:
{
  "is_name_in_message": true,
  "name": "Alice"
}

In [18]:
message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>